### Data fetch

In [16]:
TRAIN_LABELS = 'participants/train/labels/labels.csv'
TRAIN_TEXT = 'participants/train/extracted_data/extract_combined.csv'
TEST_TEXT = 'participants/test/extracted_data/extract_combined.csv'
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB


In [17]:
import pandas as pd
import numpy as np
train_labels_df = pd.read_csv(TRAIN_LABELS, usecols=['document_name','is_fitara'])
train_text_df = pd.read_csv(TRAIN_TEXT)
test_df = pd.read_csv(TEST_TEXT)

msk = np.random.rand(train_text_df.shape[0]) < 0.7

### Train Data

In [18]:
# train_y = pd.get_dummies(train_labels_df.is_fitara.values)
# train_labels_df_OHE['Yes'] = train_y.Yes
# train_labels_df_OHE = train_labels_df_OHE.drop(['is_fitara'], axis = 1)
# train_labels_df_OHE = train_labels_df.copy()
train_labels_df.is_fitara = train_labels_df.is_fitara.eq('Yes').mul(1)

In [19]:
# combine labels with text features
train_df = pd.merge(
    train_labels_df, 
    train_text_df, 
    on='document_name', 
    how='inner'
)
train_df.head()

,document_name,is_fitara,text
0,04-42RFP.pdf,0,\n \nOMB No. 0990-0115 \n \n Electronic Requ...
1,1_-_Brand_name_JOFOC_1782798.pdf,0,\n \nJustification for Other than Full and Op...
2,1_-_Brand_Name_JOFOC_for_FBO.docx,0,Justification for Other than Full and Open Com...
3,1_-_Brand_Name_Only.docx,1,Justification for Other than Full and Open Com...
4,1_-_JOFOC_-_FSSI_Limited_Source.doc,1,Attachment C\r\r\rHHS Template and Instruction...


### Split data

In [5]:
train_XY = train_df[msk]
test_XY = train_df[~msk]

### NGram TFIDF

### Test Data,  TFIDF, Count with MultinoialNB

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train_XY.text)
ngram_train = tfidf_vect_ngram.transform(train_XY.text.values)
ngram_test = tfidf_vect_ngram.transform(test_XY.text.values)

In [7]:
nb_classifier2 = MultinomialNB()
nb_classifier2.fit(ngram_train, train_XY.is_fitara)
ngram_pred = nb_classifier2.predict(ngram_test)
ngram_score = metrics.accuracy_score(test_XY.is_fitara, ngram_pred)


In [8]:
print('============= NGRAM VECTORIZER ================')
print('Loss = ',metrics.log_loss(test_XY.is_fitara, ngram_pred))
print('Score = ',ngram_score)
print('CM = \n',metrics.confusion_matrix(test_XY.is_fitara, ngram_pred, labels=[0, 1]))

============= NGRAM VECTORIZER ================
Loss =  6.60644513180034
Score =  0.8087248322147651
CM = 
 [[199  13]
 [ 44  42]]


In [9]:
ngram_pred_prob = nb_classifier2.predict_proba(ngram_test)

In [10]:
ngram_validate = tfidf_vect_ngram.transform(test_df.text.values)

In [11]:
ngram_validate_pred = nb_classifier2.predict_proba(ngram_validate)

In [12]:
predictions_df = pd.DataFrame()
predictions_df['document_name'] = test_df.document_name
predictions_df['pred_fitara'] = np.round(ngram_validate_pred[:,1], decimals= 4)
# predictions_df.to_csv('participants/predictions.csv', sep=',', index=False)

In [13]:
# predictions_df.to_csv('participants/predictions.csv', sep=',', index=False)